In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
import numpy as np
from datetime import datetime 
from my_cuda import solve_cuda64

### Sove $\mathbf{x}$ satisfying $A\mathbf{x} = b$


In [13]:
dim = 200
# dim = 20
A = np.random.randn(dim,dim).astype(np.float64)
A = np.matmul(A.T, A)
b = np.random.randn(dim).astype(np.float64)
print(A.shape, b.shape)


(200, 200) (200,)


$\mathbf{x} = A^{-1}b$

In [14]:
start_time = datetime.now()
x = np.matmul(np.linalg.inv(A), b)
time_elapsed = datetime.now() - start_time 
print('inv elapsed time:')
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

inv elapsed time:
Time elapsed (hh:mm:ss.ms) 0:00:00.762921


In [15]:
b_predict = np.matmul(A, x)
print(b_predict[0:5], '\n', b[0:5])

[ 0.80085396 -1.78442913 -1.56541704  0.49102301 -1.56689102] 
 [ 0.80085396 -1.78442913 -1.56541704  0.49102301 -1.56689102]


Use np.linalg.solve

In [16]:
start_time = datetime.now()
x1 = np.linalg.solve(A, b)
time_elapsed = datetime.now() - start_time 
print('linear system solver elapsed time:')
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

print('x=:', x1[0:5], '\n', 'prev x=', x[0:5])

linear system solver elapsed time:
Time elapsed (hh:mm:ss.ms) 0:00:00.220677
x=: [ 11.71280031  16.72438469  12.39190056   6.50830082 -11.31656441] 
 prev x= [ 11.71280031  16.72438469  12.39190056   6.50830082 -11.31656441]


Vanilla Gauss Elimination using for loops

In [18]:
WorkMat = np.concatenate([A, b.reshape([-1,1])], axis=1)

start_time = datetime.now()
for icnt1 in range(dim):   # sequential
    for icnt2 in range(dim):   # thread per icnt2
        if icnt2 == icnt1:
            continue
        curRatio = WorkMat[icnt2,icnt1]/WorkMat[icnt1,icnt1]
        for icnt3 in range(dim + 1):
            WorkMat[icnt2,icnt3] = WorkMat[icnt2,icnt3] - WorkMat[icnt1,icnt3]*curRatio
for icnt1 in range(dim):
    WorkMat[icnt1,dim] = WorkMat[icnt1,dim]/WorkMat[icnt1,icnt1]
    WorkMat[icnt1,icnt1] = 1

time_elapsed = datetime.now() - start_time 
print('Gauss Elimination #1 elapsed time:')
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))


Gauss Elimination #1 elapsed time:
Time elapsed (hh:mm:ss.ms) 0:00:11.133814


### my cuda

In [17]:
start_time = datetime.now()

x_mine = solve_cuda64(A, b)

time_elapsed = datetime.now() - start_time 
print('Gauss Elimination pycuda - my code - elapsed time:')
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))


Gauss Elimination pycuda - my code - elapsed time:
Time elapsed (hh:mm:ss.ms) 0:00:00.818804


In [9]:
print(np.matmul(A, x_mine).shape)

(20,)


In [10]:

print('x=:', x_mine[0:5], '\n', 'prev x=', x[0:5])

x=: [ 29.90541173  35.52667844   9.47146743  -7.04397768 -19.58534499] 
 prev x= [ 29.90541173  35.52667844   9.47146743  -7.04397768 -19.58534499]


In [11]:
b_predict = np.matmul(A, x_mine)
print(b_predict[0:5], '\n', b[0:5])

[-0.61574601  2.83341868  0.02200844 -1.08490053 -1.65665959] 
 [-0.61574601  2.83341868  0.02200844 -1.08490053 -1.65665959]
